In [ ]:
import flappy_bird_gymnasium
import gymnasium
import torch
from torch import nn
import torch.nn.functional as F
from collections import deque
import random
import itertools
import yaml

In [ ]:
class ReplayMemory():
    def __init__(self, maxlen, seed=None):
        self.memory = deque(maxlen=maxlen)
        if seed is not None:
            random.seed(seed)

    def append(self, transition):
        self.memory.append(transition)

    def sample(self, sample_size):
        return random.sample(self.memory, sample_size)

    def __len__(self):
        return len(self.memory)

In [14]:
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
state_dim = 12
action_dim = 2
net = DQN(state_dim, action_dim)
state = torch.randn(10, state_dim)
output = net(state)
print(output)

tensor([[ 0.1160,  0.0335],
        [-0.2501,  0.1937],
        [ 0.2432,  0.1318],
        [ 0.1034, -0.2127],
        [ 0.1952,  0.1245],
        [ 0.2403,  0.1420],
        [ 0.0785, -0.3827],
        [-0.3557, -0.1583],
        [ 0.0288, -0.0472],
        [-0.0570,  0.0486]], grad_fn=<AddmmBackward0>)


In [19]:
is_training = True

# env = gymnasium.make("FlappyBird-v0", render_mode="human", use_lidar=True)
env = gymnasium.make("CartPole-v1", render_mode=None if is_training else "human")

number_of_episodes = 1
num_actions = env.action_space.n
num_states = env.observation_space.shape[0]

rewards_per_episode = []

policy_dqn = DQN(num_states, num_actions)

if (is_training):
    memory = ReplayMemory(10000)

In [17]:
for epsiode in itertools.count():
    state, _ = env.reset()
    terminated = False
    episode_reward = 0
    while not terminated:
        # Actions:
        # 0 - do nothing
        # 1 - flap
        action = env.action_space.sample()

        # Processing:
        new_state, reward, terminated, _, info = env.step(action)

        if (is_training):
            memory.append((state, action, new_state, reward, terminated))

        state = new_state
        episode_reward += reward
        
        # Rendering:
        env.render()
        # Rewards:
        #+0.1 - every frame it stays alive
        #+1.0 - successfully passing a pipe
        #-1.0 - dying
        #−0.5 - touch the top of the screen

        # Checking if the player is still alive
    rewards_per_episode.append(episode_reward)

env.close()

/Users/richard.depierre/Documents/epitech/flappy-bird-deep-q-learning/.venv/lib/python3.11/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
